### 1. Load Python modules

In [ ]:
# Load necessary python packages.
import sys
!{sys.executable} -m pip install cufflinks > /dev/null; # Remove > /dev/null in case of errors.

from ipywidgets import interact
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cufflinks as cf
import plotly.express as px

# (Ignore the pip version warning)

### 2. Load input data

In [ ]:
# Load the values of nitrogen NH3 concentration for three different stations (Wekerom, Vredepeel and Zegveld). 
# Data are loaded as hourly values in ug/m3.
# N.B.: Do not try to hard to understand all of the code. We will mention which parts are important to understand.

df_Wekerom = pd.read_csv('../Data_IntegrationCourseSWA/Data_AQ_Wekerom_2013-2019.csv', sep=';', 
    index_col='date-time', usecols=['date-time', 'NH3'], parse_dates=['date-time'])
df_Wekerom.columns = ['NH3_W']

df_Vredepeel = pd.read_csv('../Data_IntegrationCourseSWA/Data_AQ_Vredepeel_2013-2019.csv', sep=';', 
    index_col='date-time', usecols=['date-time', 'NH3'], parse_dates=['date-time'])
df_Vredepeel.columns = ['NH3_V']

df_Zegveld = pd.read_csv('../Data_IntegrationCourseSWA/Data_AQ_Zegveld_2013-2019.csv', sep=';', 
    index_col='date-time', usecols=['date-time', 'NH3'], parse_dates=['date-time'])
df_Zegveld.columns = ['NH3_Z']

# Save the data from Wekerom, Vredepeel and Zegveld in the same data frame.
df_AQ = pd.concat([df_Wekerom, df_Vredepeel, df_Zegveld], axis=1, sort=False)

In [ ]:
# Load the meteorological data. 
# Load the velocity (u; m/s), global radiation (Rg; W/m^2), rain (Rain; mm), rain in the last three hours 
# (Rain_last3h; mm) and leaf area index (LAI; m^2/m^2).
df_meteo = pd.read_csv('../Data_IntegrationCourseSWA/Data_Meteo_Veenkampen_2013-2019.csv', sep=';', 
    index_col='date-time', usecols=['date-time','Rg','Ta','RH','u','P','smc_065','smc_125','smc_250','smc_500'],
                           parse_dates=['date-time'])
df_meteo.columns = ['Rg','Ta','RH','u','P','smc_065','smc_125','smc_250','smc_500']

### 3. Physical constants and model parameters

In [ ]:
### Constants and Parameter Settings
#--- Collect all settings here to keep overview

### Physical constants




#### Model parameters
# Set the constants needed for calculation of resistances for your land use type 
# in Kumar et al. (2001): https://link.springer.com/article/10.1007%2Fs10546-010-9559-z, Table 1; 

#--- Parameters for r_a
k     = 0.4     # - Von Karman constant
z     = 10.     # m Reference height
zd    = 0.0     # m Displacement height
z0    = 0.0001  # m Roughness length Water

#--- Parameters for r_c
rcmin = 40.0    # s/m    Minimum canopy resistance, when the stomates are fully open.
Rgl   = 100.0   # W/m2   Sensitivity to global radiation
LAI   = 2.0     # m2/m2  Leaf Area Index
rcmax = 10000.  # s/m    Maximum canopy resistance, when the stomates are closed.

# 4. NH3 deposition

### 4.1 $r_a$: Aerodynamical resistance

In [ ]:
# Calculate the aerodynamical resistance. 
# 2. Calculate the aerodynamical resistance based on the formula given in section 3.1.
ra = (np.log((z-zd)/z0))**2/(k**2*df_meteo['u'])

### 4.2 $r_b$: Boundary layer resistance

In [ ]:
rb = 5.

### 4.3 $r_c$: Canopy resistance

In [ ]:
# Write the values of your chosen parameters using the land types given in Table 1 of Kumar et al, 2001.
# Numbers should represent the values for the minimum canopy resistance (rcmin; s/m), minimum global radiation for
# photosynthesis (Rgl; W/m^2), the leaf area index (LAI, m^2/m^2), and maximum canopy resistance (rcmax; s/m). 
# Please also take a look at the Python code, it may be useful to recognise what is being done here.


# Calculate f and the stress function for solar radiation F1.
f     = 0.55*(df_meteo['Rg']/Rgl)*(2./LAI)
F1    = ((rcmin/rcmax)+f)/(1+f)

# Calculate the canopy resistance in s/m based on the formula given in the markdown-box above.
rc    = rcmin/(LAI*F1)  # s/m Hourly canopy resistance, computed from the parameters supplied above.

### 4.4 $r_t$: Total resistance

N.B.: Water surfaces do not have stomates and their $r_c$ = 0 s/m. Make sure you edit the following lines accordingly by setting rc = 0.0 or $r_t$ = $r_a$ + $r_b$

In [ ]:
###
rt = ra + rb + rc

### 4.5 $F_{NH_3}$: Ammonia deposition flux

In [ ]:
# Calculate the dry deposition rate (F_NH3; ug/m^2/s) using a gradient-resistance model for all three stations,
# i.e., for Wekerom, Vredepeel and Zegveld.
F_NH3_W = df_AQ['NH3_W']/rt
F_NH3_W.columns = ['F_NH3_W']

F_NH3_V = df_AQ['NH3_V']/rt
F_NH3_V.columns = ['F_NH3_V']

F_NH3_Z = df_AQ['NH3_Z']/rt
F_NH3_Z.columns = ['F_NH3_Z']


### 4.6 Some output options for you to tailor (figures, tables, datafiles)

In [ ]:
## Check content of a data frame
print(df_meteo.keys())
print(df_AQ   .keys())

### 4.6.1 Figure with mean seasonal cycle

In [ ]:
## Figure with mean seasonal cycle

# Calculate the inter-annual seasonal variability for all three stations. 
# Inter-annual seasonal variability can be calculated as a mean of all e.g., Januaries in our time series,
# or as a mean of all e.g., first weeks of the year.
df_mean_seasonal = df_AQ.groupby( 2*((df_AQ.index.week-1)//2 + 1)).mean()

# Plot the result (multi-year seasonal variability) for all three stations 
# (Wekerom in orange, Vredepeel in blue and Zegveld in green). 
# Plot Time [weeks] on x-axis and Concentration [um/m^3] on y-axis.
fig1 = df_mean_seasonal.iplot(asFigure=True, xTitle="Time [weeks]", yTitle="Concentration [ug/m3]", width=2)
fig1.show()

### 4.6.2 Figure with mean diurnal cycle

In [ ]:
## Mean diurnal cycle

# Calculate the multi-year daily variability in NH3 concentration for June and December for all three stations.
# Multi-year daily variability is calculated as a mean of all days in all 
# e.g., Junes and Decembers of all years in our time series.
# First, we extract June and December from our time series.
df_AQ_jun = df_AQ.loc[(df_AQ.index.month==6 )]
df_AQ_dec = df_AQ.loc[(df_AQ.index.month==12)]

# Next, we calculate the daily variability.
df_mean_jun_daily = df_AQ_jun.groupby([df_AQ_jun.index.hour]).mean()
df_mean_dec_daily = df_AQ_dec.groupby([df_AQ_dec.index.hour]).mean()

# Finally, we set up the new data (i.e., the daily variabilities) in a new matrix.
df_mean_jun_dec = pd.concat([df_mean_jun_daily, df_mean_dec_daily], axis=1, sort=False)
df_mean_jun_dec.columns = ['NH3_W_J', 'NH3_V_J', 'NH3_Z_J', 'NH3_W_D', 'NH3_V_D', 'NH3_Z_D']

# Plot the result (multi-year daily variability for June and December) for all three stations 
# (Wekerom in orange, Vredepeel in blue and Zegveld in green). 
# Results for June are given with solid lines and results for December with dotted lines. 
# Plot Time [hours] on x-axis and Concentration [um/m^3] on y-axis. 
fig2 = df_mean_jun_dec.iplot(asFigure=True, xTitle="Time [hours]", yTitle="Concentration [um/m3]", 
    colors=['orange', 'blue', 'green', 'orange', 'blue', 'green'], 
    dash=['solid', 'solid', 'solid', 'dot', 'dot', 'dot'], width=2)
fig2.show()

### 4.6.3 Table with annual means

In [ ]:
## Table with annual means

# Fill in the variables you want to output.
df_output = pd.concat([F_NH3_W, F_NH3_V, F_NH3_Z],axis=1, sort=False)
df_output.columns = ['F_NH3_W','F_NH3_V', 'F_NH3_Z']
output = df_output[['F_NH3_W','F_NH3_V', 'F_NH3_Z']].groupby([df_output[['F_NH3_W']].index.year]).mean()
print(output)

### 4.6.4 Save output as a csv file

In [ ]:
## Save flux data to csv file

# I recommend to try and make the figure in this Notebook
# - if you change anything in the notebook, you only need to run the cell to update the figure
# - in Excel you need to reimport the data and make the figure all over again.
# - anyway, you may output the data in the following way:
df_output.to_csv('../Data_IntegrationCourseSWA/MyOutput.csv', sep=';')

print('You can find the file in the filemanager in the folder \"Data_IntegrationCourseSWA\".')